In [1]:
import re, sys, dash, mpl_toolkits.basemap

import pandas as pd
import requests as r
import zipcodes as zc
import matplotlib.pyplot as plt
import dash_core_components as dcc
import dash_html_components as html

from bs4 import BeautifulSoup as bs
from dash.dependencies import Input, Output
from textwrap import dedent as d


In [2]:
from IPython import display
def show_app(app,  # type: dash.Dash
             port=9999,
             width=1000,
             height=500,
             offline=True,
             style=True,
             **dash_flask_kwargs):
    """
    Run the application inside a Jupyter notebook and show an iframe with it
    :param app:
    :param port:
    :param width:
    :param height:
    :param offline:
    :return:
    """
    url = 'http://localhost:%d' % port
    iframe = '<iframe src="{url}" width={width} height={height}></iframe>'.format(url=url,
                                                                                  width=width,
                                                                                  height=height)
    display.display_html(iframe, raw=True)
    if offline:
        app.css.config.serve_locally = True
        app.scripts.config.serve_locally = True
    if style:
        external_css = ["https://fonts.googleapis.com/css?family=Raleway:400,300,600",
                        "https://maxcdn.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css",
                        "http://getbootstrap.com/dist/css/bootstrap.min.css", ]

        for css in external_css:
            app.css.append_css({"external_url": css})

        external_js = ["https://code.jquery.com/jquery-3.2.1.min.js",
                       "https://cdn.rawgit.com/plotly/dash-app-stylesheets/a3401de132a6d0b652ba11548736b1d1e80aa10d/dash-goldman-sachs-report-js.js",
                       "http://getbootstrap.com/dist/js/bootstrap.min.js"]

        for js in external_js:
            app.scripts.append_script({"external_url": js})

    return app.run_server(debug=False,  # needs to be false in Jupyter
                          port=port,
                          **dash_flask_kwargs)


In [49]:
df = pd.read_csv('/Users/tyler/Downloads/transactions.csv')
df.drop(columns=['Category','Labels', 'Notes'], inplace=True)

In [4]:
def pull_span_zip(span_text:str, store_no:str):
    pat = '^%s'%(store_no)
    if re.match(pat, span_text):
        zipcode = re.findall(' (\d{5})', span_text)
        return zipcode[0]
    else:
        return None
    
        
def search_chip_no(store_no:str):
    chip_job_url = "https://jobs.chipotle.com/search-jobs/%s"%(str(store_no))
    cont = bs(r.get(chip_job_url).content, 'html5lib')
    spans = cont.find_all('span')
    lines = [span.get_text() for span in spans]
    for x in lines: 
        zipc = pull_span_zip(x, store_no)
        if zipc:
            break
    
    return [store_no, zipc, x]
            
def chip_txns(df:pd.DataFrame, regex):
    df = df[df['Original Description'].str.contains(regex)]
    return df

def parse_store_loc(descriptor:str):
    nos = re.findall(' (\d{4})', descriptor)
    return nos[0]

def pull_lat_long(zipcode:str):
    try:
        return [zc.matching(zipcode)[0]['lat'],
                  zc.matching(zipcode)[0]['long']]
    except:
        return [None,None]

In [50]:
# LIMIT TO CHIPOTLE TXNS
cdf = chip_txns(df, 'CHIPOTLE')
# EXTRACT STORE NO
cdf['store_no'] = cdf.loc[:,'Original Description'].apply(lambda x: parse_store_loc(x))
# RESET INDEX
cdf.reset_index(inplace=True, drop=True)

# Pull Location Info
zip_addr = cdf.store_no.apply(search_chip_no)
zip_df = pd.DataFrame(list(zip_addr), columns=['store_no','zip_code','address'])
# CONCAT zip_df cols to cdf
tdf = pd.concat([cdf, zip_df], axis=1)

# GET LAT LONS
latlons = tdf.zip_code.apply(pull_lat_long)
latlons_df = pd.DataFrame(list(latlons),columns=['lat', 'long'])
# CONCAT latlons df cols to cdf
ldf = pd.concat([tdf, latlons_df], axis=1)

In [54]:
ldf.head()

,Date,Description,Original Description,Amount,Transaction Type,Account Name,store_no,store_no,zip_code,address,lat,long
0,2/28/2018,Chipotle,CHIPOTLE 0894,10.63,debit,CHASE SAPPHIRE PREFERRED,0894,0894,80301,"0894 - 29th Street-(00894) - 1650 28th St, Ste...",40.02,-105.25
1,2/15/2018,Chipotle,CHIPOTLE 0894,18.26,debit,CHASE SAPPHIRE PREFERRED,0894,0894,80301,"0894 - 29th Street-(00894) - 1650 28th St, Ste...",40.02,-105.25
2,1/25/2018,Chipotle,CHIPOTLE 0894,12.86,debit,CHASE SAPPHIRE PREFERRED,0894,0894,80301,"0894 - 29th Street-(00894) - 1650 28th St, Ste...",40.02,-105.25
3,1/25/2018,Chipotle,CHIPOTLE 0894,5.45,debit,CHASE SAPPHIRE PREFERRED,0894,0894,80301,"0894 - 29th Street-(00894) - 1650 28th St, Ste...",40.02,-105.25
4,1/07/2018,Chipotle,CHIPOTLE 0894,10.79,debit,CHASE SAPPHIRE PREFERRED,0894,0894,80301,"0894 - 29th Street-(00894) - 1650 28th St, Ste...",40.02,-105.25


In [60]:
df = ldf

In [61]:

scl = [ [0,"rgb(5, 10, 172)"],[0.35,"rgb(40, 60, 190)"],[0.5,"rgb(70, 100, 245)"],\
    [0.6,"rgb(90, 120, 245)"],[0.7,"rgb(106, 137, 247)"],[1,"rgb(220, 220, 220)"] ]

data = [ dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lon = df['long'],
        lat = df['lat'],
        text = df['address'],
        mode = 'markers',
        marker = dict(
            size = 4,
            opacity = 0.8,
            reversescale = True,
            autocolorscale = False,
            symbol = 'circle',
            line = dict(
                width=1,
                color='rgba(102, 102, 102)'
            ),
            colorscale = scl,
            cmin = 0,
            colorbar=dict(
                title="frequency"
            )
        ))]

layout = dict(
        title = 'foobar',
        colorbar = True,
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showland = True,
            landcolor = "rgb(250, 250, 250)",
            subunitcolor = "rgb(217, 217, 217)",
            countrycolor = "rgb(217, 217, 217)",
            countrywidth = 0.5,
            subunitwidth = 0.5
        ),
    )

styles = {
    'pre': {
        'border': 'thin lightgrey solid',
        'overflowX': 'scroll'
    }
}


fig = dict( data=data, layout=layout )    

app = dash.Dash()
app.layout  = html.Div([
    dcc.Graph(id='basic-interactions', figure=fig)])


In [62]:
show_app(app_simple)

 * Running on http://127.0.0.1:9999/ (Press CTRL+C to quit)
/Users/tyler/Envs/chipy3/lib/python3.6/site-packages/dash/resources.py:36: UserWarning:

A local version of https://code.jquery.com/jquery-3.2.1.min.js is not available

/Users/tyler/Envs/chipy3/lib/python3.6/site-packages/dash/resources.py:36: UserWarning:

A local version of https://cdn.rawgit.com/plotly/dash-app-stylesheets/a3401de132a6d0b652ba11548736b1d1e80aa10d/dash-goldman-sachs-report-js.js is not available

/Users/tyler/Envs/chipy3/lib/python3.6/site-packages/dash/resources.py:36: UserWarning:

A local version of http://getbootstrap.com/dist/js/bootstrap.min.js is not available

/Users/tyler/Envs/chipy3/lib/python3.6/site-packages/dash/resources.py:36: UserWarning:

A local version of https://fonts.googleapis.com/css?family=Raleway:400,300,600 is not available

/Users/tyler/Envs/chipy3/lib/python3.6/site-packages/dash/resources.py:36: UserWarning:

A local version of https://maxcdn.bootstrapcdn.com/font-awesome/4.7.0/

In [51]:
ldf.address.value_counts()

1702 - San Francisco Centre-(01702) - 865 Market Street, Space C10, San Francisco 94103    18
1187 - Colma-(01187) - 990 Serramonte Blvd, Suite D, Colma 94014                           11
0894 - 29th Street-(00894) - 1650 28th St, Ste 1224, Boulder 80301-1013                     6
1637 - Lakeshore Plaza-(01637) - 1523 Sloat Blvd, San Francisco 94132                       6
1265 - Palo Alto-(01265) - 2675 El Camino Real, Palo Alto 94306                             5
1031 - Newark-(01031) - 34883 Newark Blvd, Newark 94560                                     4
2080 - Stonestown Galleria-(02080) - 3251 20th Ave, Spc OP183, San Francisco 94132          3
1460 - Metreon-(01460) - 121 4th St #135, San Francisco 94103                               3
0043 - Westbank-(00043) - 3300 Bee Cave Road, #670, Austin 78746                            2
0839 - 525 Market Street-(00839) - 525 Market Street, San Francisco 94105                   2
1174 - San Carlos-(01174) - 1135 Industrial Road, Suite C, S

In [52]:
ldf['Account Name'].value_counts()

CHASE SAPPHIRE PREFERRED    40
PLUS CHECKING               19
Blue Sky                    13
MASTERCARD PLATINUM          3
Name: Account Name, dtype: int64